In [30]:
import numpy as np
from math import exp, sqrt, log
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
from scipy.special import erf, erfinv    

In [31]:
def NumberofPayment(freq, T):
    nn = int((float(T) / float(freq)) * 12)
    return nn 

In [32]:
# here is the right value for the amortisation
def Amortisation(freq, T, N):
    NP = NumberofPayment(freq, T)
    Amortisation = np.zeros(NP)
    for i in range(NP):
        Amortisation[i] = float(N) / float(NP)
    return Amortisation    

In [33]:
# here is the right value for the notional value 
def Notional_Value(freq,T,N):     
    Cashflow = []
    Amortisation = []
    NP = NumberofPayment(freq, T)
    for i in range(NP):
        Amortisation.append(float(N) / float(NP))
    AA = 0
    for i in range(NP):
        Cashflow.append(N - AA)
        AA = Amortisation[i] + AA
    return Cashflow  


In [34]:
# here is the "Cubic Interpolation" to derive the forward_rate 
def Forward_Rate(freq, T, Indexant, tenor, swap_rate):
    NP = NumberofPayment(freq, T)    
    zero_rate = np.zeros(len(swap_rate))      
    count = 0   
    if len(tenor) == len(swap_rate):
        for jj in range(len(tenor) - 1):
            if tenor[jj] < 1:
                zero_rate[jj] = swap_rate[jj]
            elif tenor[jj] == 1:
                zero_rate[jj] = swap_rate[jj]
                count = jj + 1
            else:
                plus = 0
                n = len(tenor)
                for i in range(count, n, 1):                      
                    plus = plus + exp(-zero_rate[i - 1])
                    Add = swap_rate[i] * plus
                    zero_rate[i] = (-log((1 - Add) / (1 + swap_rate[i])))/tenor[i]
                             

    f2 = interp1d(tenor, zero_rate, kind = 'cubic')     
    xnew = np.linspace(tenor[0], tenor[len(tenor)-1], num = 50, endpoint = True)
    
    range_rate=int(tenor[len(tenor) - 1] * (12/freq))
    A_int_zero_rate = np.zeros(range_rate)
    C_int_zero_rate = np.zeros(range_rate) 
    new_tenor = np.zeros(len(A_int_zero_rate))     
    payment_time = float(freq) / float(12) 
    for value in range(len(A_int_zero_rate)):   
            if value==0:                             
                  new_tenor[value] = payment_time 
            else:                
                  new_tenor[value] = new_tenor[value - 1] + payment_time                        
    k = 0     
    if len(new_tenor) == len(A_int_zero_rate):
        for value in range(len(new_tenor)):          
                C_int_zero_rate[value] = f2(new_tenor[value])                          # cumulative value
                A_int_zero_rate[value] = f2(new_tenor[value]) / new_tenor[value]       # Annualized value  
                
    else:
        print ("There is something wrong!")   
    #print A_int_zero_rate, len(A_int_zero_rate)
    #print C_int_zero_rate, len(C_int_zero_rate)
    f_tenor = np.zeros(NP)
    payment_time = float(freq) / float(12) 
    for value in range(len(f_tenor)):   
            if value==0:                             
                  f_tenor[value] = payment_time 
            else:                
                  f_tenor[value] = f_tenor[value - 1] + payment_time   

    forward_rate = np.zeros(len(f_tenor))
    if len(forward_rate) == len(f_tenor):
        for value in range(len(f_tenor)):
            forward_rate[value] = A_int_zero_rate[value + Indexant] - A_int_zero_rate[value]
        #print forward_rate        
        return forward_rate  
    else:
        print ("Something is wrong!")
        return []          

In [35]:
# here is the right value for the cash fllow installment
def Cashflow_Instalment(freq, T, N, FR):
    NP = NumberofPayment(freq, T)
    Cashflow = np.zeros(NP)
    Amortisation = np.zeros(NP)
    for i in range(NP):
        Amortisation[i] = float(N) / float(NP)
    AA = 0
    for i in range(NP):
        Cashflow[i] = Amortisation[i] + (N - AA) * (exp(FR[i]) - 1)
        AA = Amortisation[i] + AA
    return Cashflow    

In [36]:
# here is the right value for the interest rate 
def Interest_Payment(freq, T, N, Indexant, FR):         # Question: DO WE NEED THE INDEXANT AS AN ARGUMENT HERE?
                                                        # Answer: as soon as we do have the FR, no we do not need 
                                                        #         the value of indexant 
    NP = NumberofPayment(freq, T)
    Cashflow = []
    Amortisation = []
    for i in range(NP):
        Amortisation.append(float(N) / float(NP))
    AA = 0
    for i in range(NP):
        Cashflow.append((N - AA) * (exp(FR[i]) - 1))
        AA = Amortisation[i] + AA
    return Cashflow   

In [37]:
# here is the test for the value of the loan
# web outputs: loan_value
# web inputs: 
#   - freq: payment_freq (1 - monthly, 3 - quarterly, 12 - annually)
#   - m: number of risk grades
#   - LGD: lgd
#   - N: loan initial amount (notional value)
#   - T: maturity in years
#   - KK: risk_class (classification grade - zero based)
#   - Indexant: interest_rate_index

def Final_LoanPrice(freq, m, LGD, N, T, KK, Indexant, prob_matrix, tenor, swap_rate):
    NP = NumberofPayment(freq, T)
    deltaT = freq / 12
    R = 1 - LGD
    FC=0.2
    LoanPrice = np.zeros((NP + 1, m))
    Probability_default_matrix = prob_matrix
    
    A = Amortisation(freq, T, N)
    forward_rate = Forward_Rate(freq, T, Indexant, tenor, swap_rate)
    NV = Notional_Value(freq, T, N)
    CI = Cashflow_Instalment(freq, T, N, forward_rate)  
    IP = Interest_Payment(freq, T, N, Indexant, forward_rate)
    
    LoanPrice[:][-1:] = CI[NP - 1]
    for i in range(NP - 1, -1, -1):
        if i > 0:
            for k in range(len(Probability_default_matrix)):
                Probability_default = Probability_default_matrix[k][len(Probability_default_matrix)] * R * (NV[i] + IP[i])
                Add = sum(LoanPrice[i + 1][k] * Probability_default_matrix[k][j] for j in range(len(Probability_default_matrix)))
                LoanPrice[i][k] = exp(-(forward_rate[i]+FC) * deltaT) * (Add + Probability_default) + CI[i - 1]
        elif i == 0:
            for k in range(len(Probability_default_matrix)):
                Probability_default = Probability_default_matrix[k][len(Probability_default_matrix)] * R * (NV[0] + IP[0])
                Add = sum(LoanPrice[1][k] * Probability_default_matrix[k][j] for j in range(len(Probability_default_matrix)))
                LoanPrice[0][k] = exp(-(forward_rate[0]+FC) * deltaT) * (Add + Probability_default)

    return LoanPrice[0][KK], CI, NP, NV, forward_rate, A, IP    

In [38]:
def f_LP(x, Value, CI, NP, T, freq):
    Sum = sum(CI[j] * exp(-x * (j + 1)) for j in range(NP))  
    return Value - Sum       

In [39]:
# web outputs: loan_return
def bisection_LP(a, b, T, freq, Value, CI, NP, tol = 0.1, maxiter = 10):
    n = 1
    while n <= maxiter:
        c = (a + b) * 0.5
        if f_LP(c, Value, CI, NP, T, freq) == 0 or abs(a - b) * 0.5 < tol:
            return c
        n += 1
        if f_LP(c, Value, CI, NP, T, freq) < 0:
            a = c
        else:
            b = c
    return c     

In [40]:
# IRR_WR we have to apply the code like that: (100% - sum), but not (Value - sum)

def f_WR(x, Value, CI, NP, T, freq):
    Sum = sum(CI[j] * exp(-x * (j + 1)) for j in range(NP))  
    return Value - Sum     


In [41]:
# web outputs: IRR_WR 
def bisection_WR(a, b, T, freq, Value, CI, NP, tol = 0.1, maxiter = 10):
    n = 1
    while n <= maxiter:
        c = (a + b) * 0.5
        if f_WR(c, Value, CI, NP, T, freq) == 0 or abs(a - b) * 0.5 < tol:
            return c
        n += 1
        if f_WR(c, Value, CI, NP, T, freq) < 0:
            a = c
        else:
            b = c
    return c 

# Here the return of this code would be IRR_WR
# IRR_WR is the interest rate return of a loan without risk which worth %100

In [42]:
# here is the test for the funding cost 
# web output: hedging_cost
def HedgingCost(freq, T, N, Indexant, NV, FR): 
    NP = NumberofPayment(freq, T)
    Time = np.zeros(NP)   
    tau = np.zeros(NP)
    discount_curve = np.zeros(NP)
    for i in range(NP):
        if i == 0:
            discount_curve[i] = exp(-FR[i])
        else:
            discount_curve[i] = exp(sum(-FR[j] for j in range(i + 1)))
    #print discount_curve
    payment_time = float(freq) / float(12)                                
    for i in range(NP):
        if i == 0:
            Time[i] = payment_time
            tau[i] = Time[i]
        else:
            Time[i] = Time[i - 1] + payment_time
            tau[i] = Time[i] - Time[i - 1]

    Add0 = sum(NV[j] * exp(FR[j]) * tau[j] * discount_curve[j] for j in range(NP))
    Add1 = sum(NV[j] * tau[j] * discount_curve[j] for j in range(NP))
    Add = log(Add0 / Add1)
    return Add 
# Here we call the return of this function as Value_HC  

In [43]:
def f_HC(x, Value_HC, CI, NP, T, freq):
    Value = 1 - Value_HC                
    Sum = sum(CI[j] * exp(-x * (j + 1)) for j in range(NP))  
    return Value - Sum 

# Here Value - sum, where Value=100% - Hedging cost amount

In [44]:
# web outputs: IRR_HC                  
def bisection_HC(a, b, T, freq, Value, CI, NP, tol = 0.1, maxiter = 10):
    n = 1
    while n <= maxiter:
        c = (a + b) * 0.5
        if f_HC(c, Value, CI, NP, T, freq) == 0 or abs(a - b) * 0.5 < tol:
            return c
        n += 1
        if f_HC(c, Value, CI, NP, T, freq) < 0:
            a = c
        else:
            b = c
    return c

# Here we will have the amount of IRR_HC 

In [45]:
def SpreadofHedgingcost(IRR_HC, IRR_WR):  
    return IRR_HC - IRR_WR

# Spread_HC= IRR_HC - IRR_WR : IRR_HC 

In [46]:
def Normal(x):
    phi = float(1) / float(2) + float(1) / float(2) * erf(x / sqrt(2))   # Question: DO WE HAVE TO REPLACE ANY OF THE VALUES ?
                                                                           # Answer: We just give the p as an input and 
                                                                           #         that is all  
    return phi     

In [47]:
def NormalInverse(p):
    phi_inv = sqrt(2) * erfinv(2 * p - 1)                      # Question: DO WE HAVE TO REPLACE ANY OF THE VALUES ?
                                                               # Answer:   We just give the p as an input and that is
                                                               #           all    
    return phi_inv    

In [48]:
def EconomicCapital(PD, N, LGD, rho, aa = 3.090232):
    NI = NormalInverse(PD)
    NN = Normal((NI + sqrt(rho) * aa) / sqrt(1 - rho) - PD)
    E = N * LGD * NN
    return E    

In [49]:
# here is the unexpected loss calculation
def Unexpectedloss(PD, N, LGD, rho, wt, wr, Nd):
    EC = EconomicCapital(PD, N, LGD, rho, aa = 3.090232)
    Sul = (wt - wr) * EC / Nd
    return Sul 

# Here we call the return of this function as Value_SU

In [50]:
def f_UL(x, Value_SU, CI, NP, T, freq):
    Value = 1 - Value_SU 
    Sum = sum(CI[j] * exp(-x * (j + 1)) for j in range(NP))  
    return Value - Sum 

# Value - sum, where Value=100% - Unexpected loss amount 

In [51]:
# web outputs: IRR_UL
def bisection_UL(a, b, T, freq, Value, CI, NP, tol = 0.1, maxiter = 10):
    n = 1
    while n <= maxiter:
        c = (a + b) * 0.5
        if f_UL(c, Value, CI, NP, T, freq) == 0 or abs(a - b) * 0.5 < tol:
            return c
        n += 1
        if f_UL(c, Value, CI, NP, T, freq) < 0:
            a = c
        else:
            b = c
    return c

In [52]:
def SpreadofUnexpectedloss(IRR_UL, IRR_WR): 
    return IRR_UL - IRR_WR

# Spread_UL= IRR_UL - IRR_WR   

In [53]:
# here is the test for the "Expected Loss" amount #
def Final_ExpectedLoss(freq, m, LGD, N, T, KK, Indexant, prob_matrix, A, forward_rate, NV, IP):
    NP = NumberofPayment(freq, T)
    deltaT = freq / 12
    FC=0.2      
    ExpectedLoss = np.zeros((NP + 1, m))

    Probability_default_matrix = prob_matrix

    ExpectedLoss[:][-1:] = 0
    for i in range(NP - 1, -1, -1):
        for k in range(len(Probability_default_matrix)):
            Probability_default = Probability_default_matrix[k][len(Probability_default_matrix)] * LGD * (NV[i] + IP[i])
            Add = sum(ExpectedLoss[i + 1][k] * Probability_default_matrix[k][j] for j in range(len(Probability_default_matrix)))
            ExpectedLoss[i][k] = exp(-(forward_rate[i]+FC) * deltaT) * (Add + Probability_default)
    return ExpectedLoss[0][KK]     

In [54]:

# here is the only difference is that the calculation does not include FC  

def FinalLoanPrice_EL(freq, m, LGD, N, T, KK, Indexant, prob_matrix, tenor, swap_rate):
    NP = NumberofPayment(freq, T)
    deltaT = freq / 12
    R = 1 - LGD 
    LoanPrice = np.zeros((NP + 1, m))
    Probability_default_matrix = prob_matrix
    
    A = Amortisation(freq, T, N)
    forward_rate = Forward_Rate(freq, T, Indexant, tenor, swap_rate)
    NV = Notional_Value(freq, T, N)
    CI = Cashflow_Instalment(freq, T, N, forward_rate)  
    IP = Interest_Payment(freq, T, N, Indexant, forward_rate)
    
    LoanPrice[:][-1:] = CI[NP - 1]
    for i in range(NP - 1, -1, -1):
        if i > 0:
            for k in range(len(Probability_default_matrix)):
                Probability_default = Probability_default_matrix[k][len(Probability_default_matrix)] * R * (NV[i] + IP[i])
                Add = sum(LoanPrice[i + 1][k] * Probability_default_matrix[k][j] for j in range(len(Probability_default_matrix)))
                LoanPrice[i][k] = exp(-forward_rate[i] * deltaT) * (Add + Probability_default) + CI[i - 1]
        elif i == 0:
            for k in range(len(Probability_default_matrix)):
                Probability_default = Probability_default_matrix[k][len(Probability_default_matrix)] * R * (NV[0] + IP[0])
                Add = sum(LoanPrice[1][k] * Probability_default_matrix[k][j] for j in range(len(Probability_default_matrix)))
                LoanPrice[0][k] = exp(-forward_rate[0] * deltaT) * (Add + Probability_default)

    return LoanPrice[0][KK] #, CI, NP, NV, forward_rate, A, IP 

In [55]:
def f_EL(x, Value_EL, CI, NP, T, freq):
    Value = 1 - Value_EL  
    Sum = sum(CI[j] * exp(-x * (j + 1)) for j in range(NP))  
    return Value - Sum      

In [56]:
# web outputs: IRR_EL   

def bisection_EL(a, b, T, freq, Value, CI, NP, tol = 0.1, maxiter = 10):
    n = 1
    while n <= maxiter:
        c = (a + b) * 0.5
        if f_EL(c, Value, CI, NP, T, freq) == 0 or abs(a - b) * 0.5 < tol:
            return c
        n += 1
        if f_EL(c, Value, CI, NP, T, freq) < 0:
            a = c
        else:
            b = c
    return c

# IRR_EL is the interest rate return of a loan with credit risk(expected loss risk)

In [57]:
def SpreadofExpectedloss(IRR_EL, IRR_WR): 
    return IRR_EL - IRR_WR

# Spread_EL= IRR_EL - IRR_WR     

In [58]:
# COMPLETE PROCESS

INP_tenor = [float(1)/float(365),float(1)/float(48),float(2)/float(48),float(1)/float(12),float(2)/float(12),float(3)/float(12),float(6)/float(12),float(9)/float(12),1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,40,50]
INP_swap_rate = [-0.0047000,-0.0037900,-0.0037100,-0.0037000,-0.0033900,-0.0032100,-0.0026800,-0.0021400,-0.0018300,-0.0017000,-0.0002000,
                 0.0014090,0.0029300,0.0043800,0.0057300,0.0069900,0.0081500,0.0091800,0.0100100,0.0109400,0.0115210,0.0121590,0.0128400,
                 0.0131550,0.0135900,0.0135900,0.0141750,0.0145400,0.0147200,0.0147200,0.0148450,0.0147200,0.0150720,0.0150800,0.0151200,
                 0.0151600,0.0151800,0.0153100,0.0153000,0.0151000]

INP_probability_matrix = [[0.97,0.02,0.01],[0.04,0.93,0.03]]

INP_payment_freq = 12   
INP_nr_risk_grades = 2
INP_lgd = 0.5
INP_loan_amount = 1
INP_maturity_years = 3
INP_risk_grade = 1
INP_indexant = 3

OUT_loan_value, OUT_CI, OUT_NP, OUT_NV, OUT_FR, OUT_amortization, OUT_IP = Final_LoanPrice(
    INP_payment_freq, INP_nr_risk_grades, INP_lgd, INP_loan_amount,
    INP_maturity_years, INP_risk_grade, INP_indexant, INP_probability_matrix,
    INP_tenor, INP_swap_rate)

print ("this is the value of the loan =", OUT_loan_value )
print ("this is the OUT_CI =", OUT_CI )
print ("this is the OUT_NP =", OUT_NP )



IRRL = bisection_LP(-10, 10, INP_maturity_years, INP_payment_freq, OUT_loan_value, OUT_CI, OUT_NP, 0.0000001, 1000)
print ("This is the return of the loan =", IRRL)


OUT_HC = HedgingCost(INP_payment_freq, INP_maturity_years, INP_loan_amount, INP_indexant, OUT_NV, OUT_FR)
print ("here is the hedging cost:", OUT_HC)



INP_PD = 0.01

# YASER: What inputs are these?
INP_rho = 0.25
INP_wt = 0.12
INP_wr = 0.02
INP_Nd = 1

OUT_UL = Unexpectedloss(INP_PD, INP_loan_amount, INP_lgd, INP_rho, INP_wt, INP_wr, 1)
print ("This is the value of the unexpected loss:", OUT_UL)



OUT_expected_loss = Final_ExpectedLoss(INP_payment_freq, INP_nr_risk_grades, INP_lgd, INP_loan_amount, 
                                       INP_maturity_years, INP_risk_grade, INP_indexant,
                                       INP_probability_matrix, OUT_amortization, OUT_FR, OUT_NV, OUT_IP)
print ("this is the value of the expected loss =", OUT_expected_loss)







# ADDED BY João Carlota



EL_AUX = FinalLoanPrice_EL(INP_payment_freq, INP_nr_risk_grades, INP_lgd, INP_loan_amount, 
                                       INP_maturity_years, INP_risk_grade, INP_indexant,
                                       INP_probability_matrix, INP_tenor, INP_swap_rate)
print ("this is the value of that will be used for IRR_EL =", EL_AUX)



IRR_EL = bisection_EL(-10, 10, INP_maturity_years, INP_payment_freq, EL_AUX, OUT_CI, OUT_NP, 0.0000001, 1000)
print ("IRR_EL =", IRR_EL)

IRR_UL = bisection_UL(-10, 10, INP_maturity_years, INP_payment_freq, OUT_UL, OUT_CI, OUT_NP, 0.0000001, 1000)
print ("IRR_UL =", IRR_UL)

IRR_HC = bisection_HC(-10, 10, INP_maturity_years, INP_payment_freq, OUT_HC, OUT_CI, OUT_NP, 0.0000001, 1000)
print ("IRR_HC =", IRR_HC)


IRR_WR = bisection_WR(-10, 10, INP_maturity_years, INP_payment_freq, 1, OUT_CI, OUT_NP, 0.0000001, 1000)
print ("IRR_WR =", IRR_WR)


SPREAD_EL = SpreadofExpectedloss(IRR_EL, IRR_WR)
print ("SPREAD_EL =", SPREAD_EL)

SPREAD_UL = SpreadofUnexpectedloss(IRR_UL, IRR_WR)
print ("SPREAD_UL =", SPREAD_UL)

SPREAD_HC = SpreadofHedgingcost(IRR_HC, IRR_WR)
print ("SPREAD_HC =", SPREAD_HC)


('this is the value of the loan =', 0.6636550693539213)
('this is the OUT_CI =', array([0.3355188 , 0.33429379, 0.33360116]))
('this is the OUT_NP =', 3)
('This is the return of the loan =', 0.21454833447933197)
('here is the hedging cost:', 0.0017058833721124874)
('This is the value of the unexpected loss:', 0.009043048479972523)
('this is the value of the expected loss =', 0.021341434418768197)
('this is the value of that will be used for IRR_EL =', 0.9706336650572612)
('IRR_EL =', 2.5189729779958725)
('IRR_UL =', 0.006258562207221985)
('IRR_HC =', 0.002561137080192566)
('IRR_WR =', 0.0017061084508895874)
('SPREAD_EL =', 2.517266869544983)
('SPREAD_UL =', 0.0045524537563323975)
('SPREAD_HC =', 0.0008550286293029785)


In [ ]:
Questions: 
    
- Do we need to calculate annulized zero_rate before one year?
- How to change the code when NV is not equal to one (1). 

Check:
    
- funding cost needs to be checked
- what is the meaning of the return of expected loss? please check it because is to high!

fututr work:

- computing portfolio risk
- using stochastic calculus
- calculating funding cost
- applying embedded option (prepayment right) 
- evaluate asset correlation 
- calculation of economic capital      

In [ ]:
# Calculating Spread that cover the expexted loss, so in order to calculate this we have to compute the
# price of the loan which assume there is no risk 

# Calculating the interest rate of return of a bond without risk   

# IRR_WR is the interest rate return of a loan without risk which worth %100
# IRR_EL is the interest rate return of a loan with credit risk(expected loss risk)
# IRR_WR we have to apply the code like that: (100% - sum), but not (Value - sum) 
# Spread_EL= IRR_EL - IRR_WR  
# Spread_HC= IRR_HC - IRR_WR : IRR_HC Value - sum, where Value=100% - Hedging cost amount
# Spread_UL= IRR_UL - IRR_WR : IRR_UL Value - sum, where Value=100% - Unexpected loss amount